In [1]:
import z3
from lib.ts import Ts
from lib.prove import prove_inductive

In [2]:
# The smart contract as a transition system.
ts = Ts('erc721')

owner = 100
sender = z3.BitVec('sender', 16)
receiver = z3.BitVec('receiver', 16)
tokenId = z3.BitVec('tokenId',16)
p = z3.BitVec('p',16)
q = z3.BitVec('q',16)
address = z3.BitVec('address',16)

# ownerOf
ownerOf, ownerOfOut = ts.add_var(z3.ArraySort(z3.BitVecSort(16), z3.BitVecSort(16)), name='ownerOf')

# exist
exists, existsOut = ts.add_var(z3.ArraySort(z3.BitVecSort(16), z3.BoolSort()), name='exists')

# Balances as an array, mapping from address to Integer.
balances, balancesOut = ts.add_var(z3.ArraySort(z3.BitVecSort(16), z3.IntSort()), name='balance')

# Allowance 
tokenApprovals, tokenApprovalsOut = ts.add_var(z3.ArraySort(z3.BitVecSort(16), z3.BitVecSort(16)),
        name='tokenApprovals')

# Operator can transfer all tokens of an account
operatorApproval, operatorApprovalOut = ts.add_var(z3.ArraySort(z3.BitVecSort(16), 
        z3.ArraySort(z3.BitVecSort(16), z3.BoolSort())), 
        name='operatorApproval')

In [3]:
ts.Init = z3.And(
    z3.ForAll(p, balances[p]==0),
    z3.ForAll(p, ownerOf[p]==0),
    z3.ForAll(p, tokenApprovals[p] == 0),
    z3.ForAll(p, exists[p] == False),
    z3.ForAll([p,q], operatorApproval[p][q]==False)
)

In [4]:
tr_mint = z3.And(sender==owner,
        receiver != 0,
        z3.Not(exists[tokenId]),
        ownerOfOut == z3.Update(ownerOf,tokenId,receiver),
        existsOut == z3.Update(exists,tokenId,True),
        balancesOut == z3.Update(balances,receiver,balances[receiver]+1),
        tokenApprovalsOut == tokenApprovals,
        operatorApprovalOut == operatorApproval
        )

tr_burn = z3.And(sender==owner,
        exists[tokenId],
        p == ownerOf[tokenId],
        balancesOut == z3.Update(balances, p, balances[p]-1),
        ownerOfOut == z3.Update(ownerOf, tokenId, 0),
        existsOut == z3.Update(exists, tokenId, False),
        tokenApprovalsOut == z3.Update(tokenApprovals, tokenId, 0),
        operatorApprovalOut == operatorApproval
        )

tr_transfer = z3.And(p==ownerOf[tokenId],
        receiver != 0,
        balancesOut == z3.Update(z3.Update(balances,p,balances[p]-1), receiver, balances[receiver]+1),
        tokenApprovalsOut == z3.Update(tokenApprovals,tokenId,0),
        existsOut == exists,
        ownerOfOut == z3.Update(ownerOf,tokenId,receiver),
        operatorApprovalOut == operatorApproval
        )

tr_approve = z3.And(sender==ownerOf[tokenId],
        tokenApprovalsOut == z3.Update(tokenApprovals, tokenId, receiver),
        existsOut == exists,
        balancesOut == balances,
        ownerOfOut == ownerOf,
        operatorApprovalOut == operatorApproval
        )

tr_setApprovalForAll = z3.And(sender != receiver,
        operatorApprovalOut == z3.Update(operatorApproval, sender, 
                    z3.Update(operatorApproval[sender], receiver, True)),
        tokenApprovalsOut == tokenApprovals,
        existsOut == exists,
        balancesOut == balances,
        ownerOfOut == ownerOf,
        )

tr_transferFrom = z3.And(
        p == ownerOf[tokenId],
        receiver != 0,
        z3.Or(sender==p, tokenApprovals[tokenId]==sender, operatorApproval[p][sender]),
        balancesOut == z3.Update(z3.Update(balances,p,balances[p]-1), receiver, balances[receiver]+1),
        tokenApprovalsOut == z3.Update(tokenApprovals,tokenId,0),
        existsOut == exists,
        ownerOfOut == z3.Update(ownerOf,tokenId,receiver),
        operatorApprovalOut == operatorApproval
)


ts.Tr = z3.Or(tr_mint, tr_burn, tr_transfer, tr_approve, tr_setApprovalForAll)

In [5]:
ts_property = z3.ForAll(p, z3.Implies(exists[p],ownerOf[p]!=0))

In [6]:
prove_inductive(ts,ts_property)

Prove init => property.
proved
Prove property is inductive.
proved
